In [ ]:
from bs4 import BeautifulSoup as bs
from splinter import Browser



In [ ]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=True)

url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = bs(html, "html.parser")

print(len(html))

In [ ]:
#collect the latest News Title and Paragraph Text


In [ ]:
#* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
#find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

In [ ]:

#* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [ ]:
#* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#* Use Pandas to convert the data to a HTML table string.

In [ ]:
### Mars Hemispheres
#* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.


In [ ]:
#click each of the links to the hemispheres in order to find the image url to the full resolution image.

In [ ]:
#* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.


In [ ]:
#* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.